In [ ]:
import pandas as pd

df_all = pd.read_csv(".data/available_TA_all_filtered.csv")
df_all.head(3)

In [ ]:
row_init = df_all[df_all["relevant"]==2].sample(1).iloc[0]
T_id = row_init["T_id"]
title = row_init["title"]
abstract = row_init["abstract"]
introduction = row_init["I_original"]
prompt = """
标题：{}
摘要：{}
引言：{}
 - 根据以上信息，总结文章所属的学科,可以返回多个学科
 - 提取其中的萜类化合物的母体化合物名，不需要立体化学以及beta或arpha，如果涉及到衍生物则返回原始化合物名。没有返回None
 - 提取这个萜类化合物生物来源的科属种，如果文中没有提到，根据你的知识填写，没有则全部返回None
EXAMPLE JSON OUTPUT:
{{
    "subject": [],
    "terpenoids": [],
    "source": [
        {{
            "family": "",
            "genus": "",
            "species": ""
        }}
    ]
}}
"""
prompt = prompt.format(title, abstract, introduction)
prompt


In [ ]:
prompt = """
标题：{}
摘要：{}
引言：{}
 - 根据以上信息，总结文章所属的学科,可以返回多个学科
 - 提取其中的萜类化合物的母体化合物名，不需要立体化学以及beta或alpha，如果涉及到衍生物则返回原始化合物名。没有返回None
 - 提取这个萜类化合物生物来源的科属种，如果文中没有提到，根据你的知识填写，没有则全部返回None
EXAMPLE JSON OUTPUT:
{{
    "subject": [],
    "terpenoids": [],
    "source": [
        {{
            "family": "",
            "genus": "",
            "species": ""
        }}
    ]
}}
"""
input_data = []
for _, row in df_all[df_all["relevant"]==2].iterrows():
    custom_id = row["T_id"]
    title = row["title"]
    abstract = row["abstract"]
    introduction = row["I_original"]
    input_data.append({
        "custom_id": custom_id,
        "llm_input": prompt.format(title, abstract, introduction)
    })

len(input_data)


In [7]:

def data_to_jsonl_volcengine(datas:list, save_file_path=".tmp.jsonl"):
    import json
    # 火山引擎要求的jsonl格式
    with open(save_file_path, "w", encoding="utf-8") as f:
        for _data in datas:
            custom_id = _data["custom_id"]
            user_input = _data["llm_input"]
            _input = {
                "custom_id": custom_id,
                "body": {
                    "messages": [
                        {"role": "system","content": ""},
                        {"role": "user","content": user_input}
                    ],
                    "max_tokens": 10000,
                    "temperature": 0
                },
            }
            f.write(json.dumps(_input, ensure_ascii=False) + "\n")
    return

# data_to_jsonl_volcengine(input_data, "subject_source_torpenoids_input.jsonl")

In [1]:
import pandas as pd
df_all = pd.read_csv(".data/available_TA_all_filtered.csv")
prompt = """
    *标题：{}
    *背景：{}
    *方法：{}
    *结果：{}
    - 根据以上信息，总结文章使用的研究方法与结果
    - 每一个方法占用一条数据，方法为通用方法，不要包含其他信息，如果文中的方法不够标准化，使用通用方法代替
    - 如果涉及到材料与设备，填写到metrial中，没有写None
    - 方法只返回关键词
    - 参考前文背景总结当前方法得到的结果
    - 只返回json格式，不要其他内容
    EXAMPLE JSON OUTPUT:
    [
        {{
            "method": "",
            "metrial": [],
            "result": ""
        }},
        {{
            "method": "",
            "metrial": [],
            "result": ""
        }}
    ]
"""
input_data = []
for _, row in df_all[df_all["relevant"]==2].iterrows():
    T_id = row["T_id"]
    title = row["title"]
    abstract = row["abstract"]
    introduction = row["I_summary"]
    if pd.isna(introduction):
        introduction = row["I_original"]
    if pd.isna(introduction):
        introduction = ""
    method = row["M_original"]
    result = row["R_original"]
    if pd.isna(result):
        result = str(row["D_original"]) + str(row["C_original"])
    input_data.append({
        "custom_id": T_id,
        "llm_input": prompt.format(title, abstract, method, result)
    })
len(input_data)

34070

In [5]:
input_data[12145]["llm_input"]

'\n    *标题：Enhance production of diterpenoids in yeast by overexpression of the fused enzyme of ERG20 and its mutant mERG20\n    *背景：Yeast has been widely used for large-scale production of terpenoids. In yeast, modifications of terpenoid biosynthetic pathways have been intensively studied. tHMG1 (encoding the catalytic domain of 3-hydroxy-3-methylglutaryl coenzyme A reductase of yeast) and UPC2-1 (the G888D mutant of UPC2 encoding a transcription factor) were integrated into yeast chromosome, and ERG9 (the squalene synthase gene of yeast) was knocked down to yield the chassis strain DH02. A F96C mutation in ERG20 (farnesyl diphosphate synthase of yeast) was conducted to obtain mERG20 which can function as a geranylgeranyl diphosphate synthase (GGPS). Then, three fused genes, including BTS1 (the yeast innate GGPS)-ERG20, ERG20-mERG20 and mERG20-ERG20, were constructed, and expressed either by the pESC-based plasmids in DH02, or by being integrated into DH02 chromosome. The highest gera

In [8]:
data_to_jsonl_volcengine(input_data, "method_extract_input_2.jsonl")


In [1]:
import pandas as pd
from tqdm import tqdm

df_kg = pd.read_csv('.data/TA_KG_info.csv')
df_kg

,T_id,subject,terpenoids,source,method
0,TA000000,"['Biotechnology', 'Pharmaceutical Sciences', '...","['carvacrol', 'eugenol', 'thymol', 'geraniol']","[{'family': 'Lamiaceae', 'genus': 'Origanum', ...","[{'method': 'database search', 'metrial': ['Ch..."
1,TA000001,"['Entomology', 'Chemistry', 'Ecology', 'Bioche...","['sesquiterpene', 'neocembrene', 'citronellal'...","[{'family': 'Formicidae', 'genus': 'Monomorium...",[{'method': 'headspace solid-phase microextrac...
2,TA000002,"['Biotechnology', 'Agriculture', 'Pharmaceutic...",['Terpenes'],"[{'family': 'None', 'genus': 'Saccharomyces', ...","[{'method': 'database search', 'metrial': ['Ch..."
3,TA000003,"['Food Science', 'Aromatherapy', 'Phytochemist...","['Terpene', 'Terpene alcohol']","[{'family': 'Asteraceae', 'genus': 'Chrysanthe...","[{'method': 'database search', 'metrial': ['Ch..."
4,TA000004,"['Environmental Science', 'Agricultural Scienc...","['pinene', 'eucalyptol', 'limonene']","[{'family': 'Pinaceae', 'genus': 'Pinus', 'spe...","[{'method': 'database search', 'metrial': ['Ch..."
...,...,...,...,...,...
34065,TA050916,"['Oncology', 'Pharmacology', 'Radiotherapy', '...",['boswellic acid'],"[{'family': 'Burseraceae', 'genus': 'Boswellia...","[{'method': 'XTT assay', 'metrial': ['glioblas..."
34066,TA050917,"['Traditional Chinese Medicine', 'Pharmacology...",['iridoid'],"[{'family': 'Rubiaceae', 'genus': 'Morinda', '...","[{'method': 'database search', 'metrial': ['Ch..."
34067,TA050918,"['Cardiology', 'Pharmacology', 'Traditional Ch...",['Ginsenoside'],"[{'family': 'Araliaceae', 'genus': 'Panax', 's...","[{'method': 'surface plasmon resonance', 'metr..."
34068,TA050919,"['Neuroscience', 'Pharmacology', 'Biochemistry...",['Dammarane'],"[{'family': 'Araliaceae', 'genus': 'Panax', 's...","[{'method': 'open field test', 'metrial': ['mi..."


In [2]:
# 总结萜类：
terpeneoids_count = {}
for _, row in df_kg.iterrows():
    T_id = row["T_id"]
    terpen_list = eval(row["terpenoids"])
    for terpene in terpen_list:
        terpene = terpene.strip()
        if terpene not in terpeneoids_count:
            terpeneoids_count[terpene] = 0
        terpeneoids_count[terpene] += 1
terpeneoids_count = dict(sorted(terpeneoids_count.items(), key=lambda item: item[1], reverse=True))
terpeneoids_count

{'pinene': 3418,
 'limonene': 3378,
 'linalool': 2148,
 'caryophyllene': 2041,
 'myrcene': 1342,
 'geraniol': 1144,
 'thymol': 826,
 'terpinene': 819,
 'carvacrol': 801,
 'cineole': 754,
 'germacrene': 751,
 'camphor': 739,
 'terpineol': 723,
 'sabinene': 716,
 'isoprene': 673,
 'isopentenyl diphosphate': 669,
 'monoterpenes': 649,
 'monoterpene': 643,
 'ocimene': 642,
 'phellandrene': 634,
 'farnesene': 628,
 'farnesyl diphosphate': 611,
 'humulene': 608,
 'sesquiterpenes': 602,
 'dimethylallyl diphosphate': 593,
 'sesquiterpene': 559,
 'geranylgeranyl diphosphate': 551,
 'camphene': 540,
 'carene': 533,
 'borneol': 506,
 '1,8-cineole': 503,
 'geranyl diphosphate': 503,
 'Limonene': 501,
 'menthol': 492,
 'cymene': 488,
 'carvone': 457,
 'nerolidol': 457,
 'terpinolene': 427,
 'squalene': 424,
 'citronellol': 405,
 'cadinene': 398,
 'Pinene': 394,
 'oleanolic acid': 390,
 'Linalool': 386,
 'p-cymene': 375,
 'abietane': 352,
 'elemene': 351,
 'citral': 349,
 'labdane': 343,
 'nerol': 3